In [1]:
import pandas as pd
import sys

sys.path.append("../scripts")
from scripts.translate_summary_json import *

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv("../datasets/augmented_clinical_notes/cleaned_df.csv")
df.head(5)

,idx,note,full_note,conversation,summary_eval
0,155216,"A a sixteen year-old girl, presented to our Ou...","A a sixteen year-old girl, presented to our Ou...","Doctor: Good morning, what brings you to the O...",{'visit motivation': 'Discomfort in the neck a...
1,133948,A 36-year old female patient visited our hospi...,A 36-year old female patient visited our hospi...,"Doctor: Hello, what brings you to the hospital...",{'visit motivation': 'Pain and restricted rang...
2,80176,A 49-year-old male presented with a complaint ...,A 49-year-old male presented with a complaint ...,"Doctor: Good morning, Mr. [Patient's Name]. I'...",{'visit motivation': 'Pain in the left proxima...
3,72232,A 47-year-old male patient was referred to the...,A 47-year-old male patient was referred to the...,"Doctor: Good morning, how are you feeling toda...",{'visit motivation': 'Recurrent attacks of pai...
4,31864,A 24-year-old Yemeni female presented to the e...,A 24-year-old Yemeni female presented to the e...,"Doctor: Good morning, how are you feeling toda...",{'visit motivation': 'Inability to walk and a ...


In [3]:
eval(df.iloc[78]["summary_eval"])

{'visit motivation': 'Large, painful swelling in the popliteal fossa in his left leg',
 'admission': [{'reason': 'Large, painful swelling in the popliteal fossa',
   'date': 'None',
   'duration': 'None',
   'care center details': 'None'}],
 'patient information': {'age': '45',
  'sex': 'Male',
  'ethnicity': 'None',
  'weight': 'None',
  'height': 'None',
  'family medical history': 'None',
  'recent travels': 'None',
  'socio economic context': 'None',
  'occupation': 'None'},
 'patient medical history': {'physiological context': 'Puncture wound by a nail 11 years ago, followed by a slowly growing mass in the popliteal fossa',
  'psychological context': 'None',
  'vaccination history': 'None',
  'allergies': 'None',
  'exercise frequency': 'None',
  'nutrition': 'None',
  'sexual history': 'None',
  'alcohol consumption': 'None',
  'drug usage': 'None',
  'smoking status': 'None'},
 'surgeries': [{'reason': 'Popliteal pseudoaneurysm',
   'Type': 'Excision of the pseudoaneurysm and pr

In [26]:
df.iloc[78]["full_note"]

"A 45-year-old man was presented in our department with a large, painful swelling in the popliteal fossa in his left leg. There has been a puncture wound by a nail 11 years ago and he resorted to another hospital. At that institution, medical treatment was given in accordance with the patient's momentary condition. Nevertheless, a slowly growing mass occurred in his popliteal fossa in a year following being subjected to the puncture wound. He resorted to cardiovascular surgery department in another hospital and Doppler ultrasonography together with arteriography was taken. A popliteal pseudoaneurysm was detected in the distal part of the popliteal artery near the bifurcation to the anterior and posterior tibial arteries and was about 4 cm in diameter (). The patient was taken into operation by cardiovascular surgeons after the angiography. He had an excision of the pseudoaneurysm and primary repair surgery to popliteal artery. After surgery, the popliteal artery was patent according to

In [4]:
summary = df[["idx","summary_eval"]].loc[:5].to_dict(orient="records")
summary

[{'idx': 155216,
  'summary_eval': "{'visit motivation': 'Discomfort in the neck and lower back, restriction of body movements, inability to maintain an erect posture, and requiring assistance in standing and walking.', 'admission': [{'reason': 'None', 'date': 'None', 'duration': 'None', 'care center details': 'None'}], 'patient information': {'age': 'Sixteen years old', 'sex': 'Female', 'ethnicity': 'None', 'weight': 'None', 'height': 'None', 'family medical history': 'None', 'recent travels': 'None', 'socio economic context': 'None', 'occupation': 'None'}, 'patient medical history': {'physiological context': 'None', 'psychological context': 'Diagnosed with bipolar affective disorder at the age of eleven, first episode was that of mania.', 'vaccination history': 'None', 'allergies': 'None', 'exercise frequency': 'None', 'nutrition': 'None', 'sexual history': 'None', 'alcohol consumption': 'None', 'drug usage': 'None', 'smoking status': 'None'}, 'surgeries': [{'reason': 'None', 'Type':

In [15]:
translate_json(summary=eval(summary[4]["summary_eval"]), id=summary[0]["idx"])

['\nPatient Information:',
 'The patient is 24 years Female.',
 '\nVisit Motivation:',
 'The patient seeks medical attention due to Inability to walk and a long history of osteomalacia.',
 '\nAdmission: No admission details provided.',
 '\nMedical History:',
 'The patient reported None.',
 '\nSymptoms:',
 '  * Inability to walk in the Legs for the past Last 4 years. Associated with severe weakness and bone pain',
 '\nTreatments:',
 '  * The patient underwent Calcium and vitamin D for Osteomalacia. No improvement.',
 '  * The patient underwent Calcitriol for Osteomalacia. No improvement, eventually stopped all treatments.',
 '\nDischarge:',
 '  * The patient was discharged due to None.',
 '  * Referral to None recommended.',
 '  * Follow-up plan: None.',
 '  * Discharge summary: None.']

In [17]:
eval(summary[4]["summary_eval"]).keys()

dict_keys(['visit motivation', 'admission', 'patient information', 'patient medical history', 'surgeries', 'symptoms', 'medical examinations', 'diagnosis tests', 'treatments', 'discharge'])

In [12]:
summary[0]["idx"]

155216

In [99]:
section_headers_dict = {k:set() for k in ['admission',
 'diagnosis tests',
 'discharge',
 'medical examinations',
 'patient information',
 'patient medical history',
 'surgeries',
 'symptoms',
 'treatments',
 'visit motivation']}

section_headers_dict

{'admission': set(),
 'diagnosis tests': set(),
 'discharge': set(),
 'medical examinations': set(),
 'patient information': set(),
 'patient medical history': set(),
 'surgeries': set(),
 'symptoms': set(),
 'treatments': set(),
 'visit motivation': set()}

In [100]:
for summary in df[["idx","summary_eval"]].to_dict(orient="records"):
    idx=summary["idx"]
    eval_summary = eval(summary["summary_eval"])
    try:
        for key in section_headers_dict.keys():
            if key in eval_summary:
                if isinstance(eval_summary[key], dict):
                    section_headers_dict[key] = section_headers_dict[key].union(set(eval_summary[key].keys()))
    
                elif isinstance(eval_summary[key], list) and len(eval_summary[key])>1:    
                    section_headers_dict[key] = section_headers_dict[key].union(set(eval_summary[key][0].keys()))

    except Exception as e:
        
        print(e, idx)
        print(key, eval_summary[key])

'str' object has no attribute 'keys' 77052
visit motivation ['Treatment of a chronic obstructive pulmonary disease (COPD) exacerbation', 'Complaining of diffuse abdominal pain, increasing abdominal girth, and anterior wall bruising', 'Worsening dyspnea and management of atrial fibrillation with rapid ventricular rate and COPD exacerbation']


In [111]:
section_headers_dict

{'admission': {'care center details', 'date', 'duration', 'reason'},
 'diagnosis tests': {'condition',
  'details',
  'result',
  'severity',
  'test',
  'time'},
 'discharge': {'discharge summary', 'follow up', 'reason', 'referral'},
 'medical examinations': {'details', 'name', 'result'},
 'patient information': {'age',
  'ethnicity',
  'family medical history',
  'height',
  'occupation',
  'recent travels',
  'sex',
  'socio economic context',
  'weight'},
 'patient medical history': {'alcohol consumption',
  'allergies',
  'drug usage',
  'exercise frequency',
  'nutrition',
  'physiological context',
  'psychological context',
  'sexual history',
  'smoking status',
  'vaccination history'},
 'surgeries': {'Type', 'details', 'outcome', 'reason', 'time'},
 'symptoms': {'behaviours affecting the symptom',
  'details',
  'intensity of symptom',
  'location',
  'name of symptom',
  'temporalisation',
  'time'},
 'treatments': {'details',
  'dosage',
  'duration',
  'frequency',
  'nam

In [110]:
for key, pairs in section_headers_dict.items():
    print(key, pairs)

admission {'reason', 'date', 'care center details', 'duration'}
diagnosis tests {'result', 'severity', 'condition', 'test', 'time', 'details'}
discharge {'follow up', 'referral', 'reason', 'discharge summary'}
medical examinations {'result', 'name', 'details'}
patient information {'socio economic context', 'recent travels', 'age', 'sex', 'weight', 'height', 'occupation', 'ethnicity', 'family medical history'}
patient medical history {'psychological context', 'drug usage', 'allergies', 'smoking status', 'nutrition', 'physiological context', 'sexual history', 'exercise frequency', 'alcohol consumption', 'vaccination history'}
surgeries {'outcome', 'reason', 'Type', 'time', 'details'}
symptoms {'name of symptom', 'intensity of symptom', 'behaviours affecting the symptom', 'location', 'time', 'temporalisation', 'details'}
treatments {'reason for taking', 'reaction to treatment', 'name', 'related condition', 'duration', 'dosage', 'time', 'frequency', 'details'}
visit motivation set()


In [102]:
print(df.iloc[5]["summary_eval"])

{'visit motivation': 'Presented with dyspnea and fever', 'admission': [{'reason': 'Dyspnea and fever, and to perform surgical repair of diaphragmatic defect', 'date': 'None', 'duration': '5 days postoperatively', 'care center details': 'None'}], 'patient information': {'age': '24-day-old', 'sex': 'Female', 'ethnicity': 'None', 'weight': 'None', 'height': 'None', 'family medical history': 'None', 'recent travels': 'None', 'socio economic context': 'None', 'occupation': 'None'}, 'patient medical history': {'physiological context': 'Normal Apgar score, no resuscitation required at birth', 'psychological context': 'None', 'vaccination history': 'None', 'allergies': 'None', 'exercise frequency': 'None', 'nutrition': 'None', 'sexual history': 'None', 'alcohol consumption': 'None', 'drug usage': 'None', 'smoking status': 'None'}, 'surgeries': [{'reason': 'Left-sided diaphragmatic defect with herniation of bowel and kidney', 'Type': 'Surgical repair of diaphragmatic defect', 'time': 'None', 'o

In [105]:
translate_admission(eval_summary)

['\nAdmission:',
 'The patient was admitted for Acute onset crushing substernal chest pressure and dyspnea with profuse sweating following 1-week-long febrile illness with malaise on None and stayed for None at Presented from an outlier hospital to our emergency department.']

In [109]:
eval('None')

In [113]:
df.iloc[0]

idx                                                        155216
note            A a sixteen year-old girl, presented to our Ou...
full_note       A a sixteen year-old girl, presented to our Ou...
conversation    Doctor: Good morning, what brings you to the O...
summary_eval    {'visit motivation': 'Discomfort in the neck a...
Name: 0, dtype: object

In [ ]:
{
    "admission": {
        "reason": "complaints of discomfort in the neck and lower back, restriction of body movements",
        "date": "four months ago",
        "care center details": "Outpatient department",
        "duration": "two years and seven months"
    },
    "diagnosis tests": [
        {
            "result": "negative",
            "severity": "not mentioned",
            "condition": "dystonia",
            "test": "MRI scan of the brain, serum and urine copper levels, slit lamp microscopy for the KF ring, complete blood count, TLC, DLC, and USG of the abdomen, thyroid function tests",
            "time": "during the course of the illness",
            "details": "did not reveal any abnormalities"
        }
    ],
    "discharge": {
        "follow up": "none mentioned",
        "referral": "none mentioned",
        "reason": "mood symptoms remitted",
        "discharge summary": "psychotropic medications were stopped, patient put on tablet tetrabenazine"
    },
    "medical examinations": [
        {
            "result": "improved slightly",
            "name": "abnormal contraction of the neck muscles and the abnormal positioning of the head",
            "details": "improved slightly, but movements failed to disappear completely"
        }
    ],
    "patient information": {
        "socio economic context": "not mentioned",
        "recent travels": "not mentioned",
        "age": "sixteen years old",
        "sex": "female",
        "weight": "not mentioned",
        "height": "not mentioned",
        "occupation": "not mentioned",
        "ethnicity": "not mentioned",
        "family medical history": "not mentioned"
    },
    "patient medical history": {
        "psychological context": "bipolar affective disorder",
        "drug usage": "olanzapine, trihexyphenidyl, propranolol, phenargan, sodium valproate, escitalopram, lithium, eltroxin, tetrabenazine",
        "allergies": "not mentioned",
        "smoking status": "not mentioned",
        "nutrition": "not mentioned",
        "physiological context": "hypothyroidism",
        "sexual history": "not mentioned",
        "exercise frequency": "not mentioned",
        "alcohol consumption": "not mentioned",
        "vaccination history": "not mentioned"
    },
    "surgeries": [],
    "symptoms": [
        {
            "name of symptom": "discomfort in the neck and lower back, restriction of body movements",
            "intensity of symptom": "varying intensity over the first three years of illness",
            "behaviours affecting the symptom": "maintaining specific postures, needing assistance in standing and walking, requiring help with daily chores",
            "location": "neck and lumbar region",
            "time": "past four months",
            "temporalisation": "associated with the introduction of olanzapine tablets",
            "details": "sustained and abnormal contraction of the neck muscles, sideways bending of the back, abnormal positioning of the head"
        }
    ],
    "treatments": [
        {
            "reason for taking": "control exacerbated mental illness",
            "reaction to treatment": "sustained abnormal contraction of the neck muscles",
            "name": "olanzapine",
            "related condition": "bipolar affective disorder",
            "duration": "seven years",
            "dosage": "2.5–10 mg per day",
            "time": "over the past seven years",
            "frequency": "daily",
            "details": "aggravated abnormal movement and posturing of the neck"
        },
        {
            "reason for taking": "stabilization of mood",
            "reaction to treatment": "amenorrhea",
            "name": "sodium valproate",
            "related condition": "mood stabilization",
            "duration": "around two years",
            "dosage": "1000 mg per day",
            "time": "during this period",
            "frequency": "daily",
            "details": "shifted to tablet lithium 450 mg per day due to valproate-induced menstrual disturbance"
        },
        {
            "reason for taking": "response to tetrabenazine",
            "reaction to treatment": "improvement in abnormal movements of the muscles of the neck and back, reduced pain and discomfort in the back and neck",
            "name": "tetrabenazine",
            "related condition": "dystonia",
            "duration": "current treatment",
            "dosage": "built up to 75 mg per day",
            "time": "current treatment",
            "frequency": "in divided doses",
            "details": "response shown with improvement in abnormal movements"
        }
    ],
    "visit motivation": "treatment of dystonia caused by olanzapine"
}